In [15]:
import pandas as pd
df = pd.read_csv('Melanoma_ExpressionData.csv', index_col=0)
df.head()


,gene_name,gene_length,patient_1,patient_2,patient_3,patient_4,patient_5,patient_6,patient_7,patient_8,patient_9,patient_10
0,A1BG,3931,1272.36,452.96,288.06,400.11,420.46,877.59,402.77,559.20,269.59,586.66
1,A1CF,2409,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
2,A2BP1,5897,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
3,A2LD1,2825,164.38,552.43,201.83,165.12,95.75,636.63,241.56,30.82,105.44,239.19
4,A2ML1,6303,27.00,0.00,0.00,0.00,8.00,0.00,1.00,763.00,0.00,0.00


In [17]:

samples = df.to_dict(orient = 'list')
samples['patient_1']

[1272.36,
 0.0,
 0.0,
 164.38,
 27.0,
 65232.35,
 190.0,
 5.0,
 0.0,
 2449.0,
 0.0,
 1571.0,
 0.0,
 0.0,
 0.0,
 0.0,
 120.0,
 1716.0,
 1799.0,
 8729.0,
 3.0,
 2807.83,
 1822.0,
 12929.0,
 1845.0,
 235.0,
 590.0,
 5927.0,
 1571.49,
 41.0,
 19.0,
 90.89,
 1.0,
 20.0,
 16.0,
 423.0,
 972.0,
 277.0,
 31.0,
 364.0,
 4.0,
 924.0,
 1.0,
 21.0,
 618.0,
 0.0,
 10.0,
 390.0,
 1557.0,
 2033.72,
 673.0,
 4757.0,
 1006.64,
 1599.0,
 2.0,
 0.0,
 0.0,
 2846.0,
 598.0,
 22.0,
 106.0,
 1999.0,
 1.0,
 15.0,
 15.0,
 1.0,
 52.0,
 15769.0,
 2.0,
 728.0,
 1042.0,
 2301.0,
 4789.0,
 5477.0,
 4693.0,
 119.0,
 299.0,
 12.0,
 74.0,
 0.0,
 2263.0,
 3428.0,
 57.0,
 5497.0,
 282.0,
 896.0,
 9210.0,
 588.0,
 80.0,
 4770.0,
 243.0,
 3073.0,
 720.0,
 1596.0,
 499.0,
 2172.08,
 2455.0,
 46.0,
 75.0,
 3447.0,
 4465.0,
 959.0,
 138.0,
 396.0,
 0.0,
 0.0,
 0.0,
 8807.0,
 1450.0,
 1566.0,
 536.0,
 3542.0,
 3284.0,
 2602.0,
 1833.0,
 1816.0,
 902.0,
 719.0,
 2491.31,
 0.0,
 2030.0,
 998.0,
 2382.0,
 13805.97,
 38.0,
 40.0,

### Normalizing Over Samples and Genes: RPKM

One of the simplest normalization methods for RNAseq data is RPKM: reads per
kilobase transcript per million reads.
RPKM puts together the ideas of normalizing by sample and by gene.
When we calculate RPKM, we are normalizing for both the library size (the sum of each column)
and the gene length.

To work through how RPKM is derived, let's define the following values:

- $C$ = Number of reads mapped to a gene
- $L$ = Exon length in base-pairs for a gene
- $N$ = Total mapped reads in the experiment

First, let's calculate reads per kilobase.

Reads per base would be:
$\frac{C}{L}$

The formula asks for reads per kilobase instead of reads per base.
One kilobase = 1,000 bases, so we'll need to divide length (L) by 1,000.

Reads per kilobase would be:

$\frac{C}{L/1000}  = \frac{10^3C}{L}$

Next, we need to normalize by library size.
If we just divide by the number of mapped reads we get:

$ \frac{10^3C}{LN} $

But biologists like thinking in millions of reads so that the numbers don't get
too small. Counting per million reads we get:

$ \frac{10^3C}{L(N/10^6)} = \frac{10^9C}{LN}$


In summary, to calculate reads per kilobase transcript per million reads:
$RPKM = \frac{10^9C}{LN}$

Now let's implement RPKM over the entire counts array.

In [30]:
C = samples['patient_1'][0]
N = sum(samples['patient_1'])
L = samples['gene_length'][0]
normed = 1e9 * C / (N*L)
print(f" original value = {samples['patient_1'][0]}, normed = {normed}")


 original value = 1272.36, normed = 7.79183928027979


In [33]:
def rpkm(data_dict, patient, number):
    """
    Function to calculate the reads per
    kilobase transcript per million reads
    """
    C = data_dict[patient][number]
    N = sum(data_dict[patient])
    L = data_dict['gene_length'][number]
    return 1e9 * C / (N*L)





In [36]:
norm_patient_1 = []
for number in  range(len(samples['patient_1'])):
    norm_value = rpkm(samples, 'patient_1', number)
    norm_patient_1.append(norm_value)



In [49]:
def rpkm_list(data_dict, patient):
    """
      Function to calculate the reads per
    kilobase transcript per million reads
    across a list of values taken from a dictionary
    """
    norm_patient = []
    for number in  range(len(data_dict[patient])):
        C = data_dict[patient][number]
        N = sum(data_dict[patient])
        L = data_dict['gene_length'][number]
        norm_patient.append(1e9 * C / (N*L))
    return norm_patient


In [50]:
norm_samples = []
for patient in list(samples.keys())[2:]:
    norm_patient = rpkm_list(samples, patient)
    norm_samples.append(norm_patient)
len(norm_samples)


10

In [58]:
norm_samples[1]

[2.2671351902006864,
 0.0,
 0.0,
 3.8475069485631614,
 0.0,
 261.0656886698351,
 1.8988815733061841,
 0.0,
 0.0,
 23.37241551345904,
 0.0,
 3.500424617900809,
 0.0,
 0.0,
 0.0,
 0.012251099551233194,
 3.120342373401422,
 13.773331841321077,
 1.819661999650596,
 54.11462303721325,
 0.018561571584226895,
 16.535493368795827,
 11.861359133485504,
 59.607531201602455,
 21.479517945616205,
 2.923258634709926,
 4.949691102435451,
 49.72960158573745,
 0.9534366169539418,
 0.7466579225566056,
 0.02783710509236065,
 3.0555955146731515,
 0.004200078104233218,
 0.04557043052017134,
 0.0036154476073650328,
 4.9159529334715,
 3.712460399061487,
 0.5090554254480699,
 0.0022902183540077416,
 1.0159031439265727,
 0.1283039782846621,
 1.4073280455318697,
 17.9916055689332,
 0.4897336921916982,
 12.133760959784505,
 0.007986712352052165,
 1.1428777183572216,
 2.1832470190333324,
 5.408703103362376,
 4.680065869778523,
 12.102144672915935,
 8.083128676754168,
 1.7133746758708552,
 11.347688762564442,
 0.

In [51]:
max_count = norm_samples[0][0]
for count in norm_samples[0]:
    if count > max_count:
        max_count = count
print(max_count)

8080.651412016943


In [66]:
max_count = 0
for number, count in enumerate(norm_samples[1]):
    if count > max_count:
        max_count = count
        max_number = number
print(max_count, samples['gene_name'][max_number])

4155.57090298624 RPS18


In [67]:
def find_max_gene(norm_sample):
    max_count = 0
    for number, count in enumerate(norm_sample):
        if count > max_count:
            max_count = count
            max_number = number
    print(max_count, samples['gene_name'][max_number])

for norm_sample in norm_samples:
    find_max_gene(norm_sample)

8080.651412016943 GAPDH
4155.57090298624 RPS18
5793.638751761916 SILV
8787.657077866417 TMSL3
6060.259687819573 CD74
4949.847684159235 SILV
7548.278319135107 B2M
5187.075405230705 GAPDH
3782.999219714893 GAPDH
4322.089921441419 RPLP1


In [65]:
max(norm_samples[1])

4155.57090298624